# Example-06: Octupole element factory

In [1]:
# In this example octupole factory is illustrated

In [2]:
import jax
from jax import jit
from jax import jacrev

from elementary.util import ptc
from elementary.util import beta
from elementary.octupole import octupole_factory

jax.numpy.set_printoptions(linewidth=256, precision=12)

In [3]:
# Set data type

jax.config.update("jax_enable_x64", True)

In [4]:
# Set device

device, *_ = jax.devices('cpu')
jax.config.update('jax_default_device', device)

In [5]:
# Set initial condition

(q_x, q_y, q_s) = qs = jax.numpy.array([-0.01, 0.005, 0.001])
(p_x, p_y, p_s) = ps = jax.numpy.array([0.001, 0.001, -0.0001])
x = jax.numpy.hstack([qs, ps])

In [6]:
# Define generic sextupole element

gamma = 10**6
element = jit(octupole_factory(beta=beta(gamma), gamma=gamma, order=2**1, iterations=100, settings=dict(ns=2**1)))

In [7]:
# Compare with PTC

length = jax.numpy.float64(0.2)
kn = jax.numpy.float64(-100.0)
ks = jax.numpy.float64(+500.0)

print(res := element(x, length, kn, ks))
print(ref := ptc(x, 'octupole', {'l': float(length), 'k3': float(kn), 'k3s': float(ks)}, gamma=gamma))
print(jax.numpy.allclose(res, ref))

[-0.009797768862  0.005199205032  0.000999798545  0.001022114248  0.000992148146 -0.0001        ]
[-0.009797768862  0.005199205032  0.000999798545  0.001022114248  0.000992148146 -0.0001        ]
True


In [8]:
# Differentiability

print(jacrev(element)(x, length, kn, ks))
print()

print(jacrev(element, 1)(x, length, kn, ks))
print()

[[ 9.995697830497e-01  4.656632792030e-04  0.000000000000e+00  1.999915493270e-01  3.093502378318e-05 -2.022534219311e-04]
 [ 4.656649770852e-04  1.000430351571e+00  0.000000000000e+00  3.093510854154e-05  2.000492660272e-01 -1.992850675588e-04]
 [-2.659895940131e-08 -9.006308190735e-07  1.000000000000e+00 -2.022529821769e-04 -1.992852940463e-04  4.030115086891e-07]
 [-4.326806733307e-03  4.608973281252e-03  0.000000000000e+00  9.995646349961e-01  4.562292899751e-04 -1.639275718681e-08]
 [ 4.609037834467e-03  4.329471623181e-03  0.000000000000e+00  4.562327538090e-04  1.000435498762e+00 -8.939564944818e-07]
 [ 1.882573265034e-27  5.786541258829e-28  0.000000000000e+00  1.104032840606e-26 -2.209639763016e-26  1.000000000000e+00]]

[ 1.022217507043e-03  9.922483777202e-04 -1.014742222594e-06  1.106308301324e-04 -3.477926503173e-05 -1.217281183102e-28]

